In [2]:
import sys

import json
import pickle
import pandas
import torch
import asyncio
import numpy as np 
import pandas as pd
import seaborn as sns   
from matplotlib import pyplot as plt
sys.path.append("/home/v-oostapenko/dev/mttl")    
from scipy.stats import entropy as calc_entropy  
from mttl.cluster_tuning.encodings import ClusterInfos
from mttl.datamodule.alpaca_data_module import AlpacaDataModule
from finetune_llama import Config
from mttl.cluster_tuning.cluster_reader import ClusterResult
from inst_follow.utils import load_model, TopicRouter,disable_torch_init
device = "cuda" if torch.cuda.is_available() else "cpu"
# import matplotlib.pyplot as plt
# import seaborn as sns

import json
import os 
import sys 
import numpy as np
import click
import json
import tqdm
import copy
import torch
import datasets
from types import SimpleNamespace
sys.path.append("/home/v-oostapenko/dev/mttl")
from inst_follow.models.clm import CLM  
from transformers import LlamaTokenizer  
from mttl.models.poly import get_selector     
from mttl.utils import hash_example
from mttl.models.modify_model import modify_transformer  
from finetune_llama import parse_config, Config
from inst_follow.utils import load_model, TopicRouter,disable_torch_init
from mttl.cluster_tuning.cluster_reader import ClusterResult
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM
import openai
openai.api_key = "f48b5a4f15dc4e58991738ab066ba465"

/anaconda/envs/comp_3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /anaconda/envs/comp_3.9/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /anaconda/envs/comp_3.9/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/anaconda/envs/comp_3.9/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /anaconda/envs/comp_3.9 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/anaconda/envs/comp_3.9/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/extras/CUPTI/lib64')}
  warn(msg)


In [82]:

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned")
# clsuter results
from mttl.cluster_tuning.cluster_reader import ClusterResult


100%|██████████| 1/1 [00:00<00:00, 612.66it/s]


In [83]:
# Training set:
# - only low entropy
# - both, low and high
# for both we need test examples with low entropy, i.e. per cluster
# for both we need train examples with high entropy

In [84]:
# per cluster       
example_to_ids_path="/home/v-oostapenko/dev/mttl/inst_follow/cluster_infos/atlas_by_instr_text-embedding-ada-002_ldalayer1.pkl"
cluster_infos = ClusterInfos.load(example_to_ids_path)
cluster_probs=np.array(cluster_infos.cluster_dists)    
cluster_probs=cluster_probs[:,cluster_probs.sum(0).nonzero()].squeeze()
entr = calc_entropy(cluster_probs, axis=-1)
max_entropy = np.log2(cluster_probs.shape[-1])
entr = entr / max_entropy
example_to_distances = dict(
                zip(
                    cluster_infos.hashes,     
                    list(cluster_infos.cluster_dists),
                )
)
example_to_indexes = dict(
    zip(
        cluster_infos.hashes,
        list(np.arange(len(cluster_infos.hashes)))))

In [85]:
low_entr_idxs = np.where(entr <= 0.1)[0]
high_entr_idxs = np.where(entr > 0.4)[0]

In [86]:
# keep the same test indexes, the ones selected for hardened<0.1
all_test_idxs = [3980, 6711, 16982, 17926, 40191, 30227, 49118, 30548, 38271, 26759, 37694, 13913, 46710, 16674, 3675, 31906, 2799, 23876, 28466, 14035, 22012, 24387, 21521, 27939, 2043, 20603, 46368, 13704, 13272, 6016, 21698, 9891, 34977, 28076, 43124, 7811, 25055, 7802, 31354, 33302, 29795, 17481, 12953, 4649, 3915, 35666, 1603, 26121, 30990, 18703, 20933, 25410, 42204, 3793, 42671, 40175, 38941, 31135, 33771, 51594, 47777, 17539, 36980, 30799, 49750, 32934, 28548, 7996, 47107, 35794, 2899, 14790, 6182, 12253, 40727, 44471, 18961, 37568, 12287, 44793, 28804, 37067, 42720, 41095, 48656, 16375, 31386, 3630, 15057, 12354, 40577, 29961, 3772, 19445, 19350, 5299, 44799, 13585, 49913, 45969, 33316, 15360, 10379, 10025, 44823, 51293, 4153, 7911, 17200, 50700, 40110, 44546, 51065, 20887, 31595, 42465, 42534, 11718, 26180, 35937, 17069, 23034, 49238, 30352, 45784, 33509, 25016, 34009, 46116, 48723, 25519, 4215, 40015, 50611, 25007, 26655, 33264, 29305, 22967, 3766, 15157, 41910, 39068, 33448, 41697, 39971, 16169, 3207, 37651, 17163, 5621, 19288, 39294, 11623, 45421, 41711, 2605, 37056, 23651, 13258, 20853, 15224, 23338, 15651, 38162, 24064, 23197, 31372, 38815, 48886, 37545, 15700, 5348, 50599, 34616, 36707, 17287, 19877, 4060, 31973, 13978, 23740, 11327, 6361, 3987, 2820, 35879, 9011, 44812, 18407, 48142, 50266, 5640, 7960, 303, 42248, 36975, 42165, 7876, 46954, 11180, 142, 20943, 21661, 25142, 33391, 49767, 43810, 41322, 9466, 28838, 6145, 30546, 39596, 8406, 19824, 14182, 49693, 2766, 36710, 44582, 10100, 14212, 51503, 15717, 50666, 32101, 14321, 48293, 13987, 12159, 29705, 42857, 28593, 23942, 8213, 41915, 19807, 6032, 5388, 46797, 45790, 51584, 23768, 21174, 35273, 46267, 36722, 33507, 47732, 8850, 43061, 20057, 15917, 19793, 43251, 4695, 22137, 2089, 33704, 888, 20740, 43494, 18228, 22007, 29777, 29533, 22195, 47634, 34863, 6717, 18087, 28580, 28074, 51352, 48849, 18728, 25090, 28611, 22848, 37995, 12018, 2357, 41638, 21184, 38587, 36761, 34552, 41796, 48169, 51549, 51524, 26202, 29983, 25763, 3477, 2386, 9815, 29808, 40765, 25408, 12710, 18566, 33960, 23798, 34363, 17485, 33847, 8303, 34648, 34517, 40849, 45320, 49826, 28591, 22131, 42407, 28310, 31288, 1332, 38410, 19079, 6867, 18320, 24082, 50523, 17715, 5246, 3296, 25192, 4657, 40610, 17978, 24838, 49880, 28053, 19753, 28986, 34054, 51560, 51517, 10605, 20230, 24763, 45370, 22049, 6804, 29865, 32648, 40680, 35613, 26581, 27784, 24761, 40424, 30829, 24472, 38274, 37277, 32354, 50310, 23536, 4864, 47459, 34385, 42595, 14908, 37935, 46282, 30301, 48716, 12824, 9497, 25137, 41275, 14594, 12519, 21885, 34142, 10548, 27683, 1583, 12319, 30256, 46616, 1124, 3887, 18260, 29885, 45817, 46524, 13542, 41758, 696, 47285, 50506, 44605, 3336, 43568, 33820, 25011, 42802, 9759, 17209, 14743, 43645, 2733, 12000, 46129, 34290, 15433, 42750, 26740, 13335, 26184, 48620, 10155, 20832, 34596, 6030, 31902, 44739, 47307, 33934, 1469, 27855, 12225, 38926, 18078, 10008, 5181, 16172, 21233, 5954, 39710, 13532, 6856, 8466, 17977, 44358, 42016, 27322, 19914, 33504, 7464, 8588, 33189, 49252, 11231, 15763, 19989, 35433, 20801, 50277, 46399, 45873, 27174, 31761, 31882, 46855, 4698, 39698, 20656, 2661, 43120, 16874, 14342, 1050, 18993, 32231, 28631, 15070, 16265, 27397, 755, 51084, 29040, 35479, 23164, 34530, 9567, 4353, 32834, 9994, 9828, 46655, 51121, 9788, 46601, 34417, 1422, 6433, 29693, 2012, 9514, 8610, 21270, 4994, 4219, 47540, 48928, 30124, 20438, 45763, 16273, 9534, 47786, 9059, 16348, 10143, 40786, 23295, 44704, 29557, 22582, 43795, 11247, 21768, 31936, 23839, 45224, 2651, 28692, 1049, 3439, 27263, 40742, 21612, 25026, 3829, 37330, 14516, 2281, 21958, 31967, 45755, 37243, 16351, 25632, 49105, 41782, 41323, 25512, 34505, 12457, 44246, 40040, 13428, 37027, 9384, 2648, 27073, 5879, 8460, 39263, 27895, 10672, 17546, 51312, 49353, 30492, 13796, 35632, 7180, 14917, 42370, 32449, 10264, 25997, 19468, 37191, 22690, 22378, 48874, 28988, 49455, 35241, 49459, 11128, 26926, 39142, 28190, 48974, 29896, 6118, 23365, 10411, 51164, 9900, 18312, 14303, 5356, 14106, 37684, 27920, 14098, 46935, 16430, 19436, 43673, 41077, 33603, 33425, 5899, 9670, 37318, 40097, 46754, 18076, 10093, 3906, 4586, 51661, 8445, 14928, 1262, 2017, 3697, 31633, 14161, 47707, 48980, 48041, 8399, 17953, 31495, 33675, 18365, 8431, 29664, 9931, 2658, 22804, 33292, 42839, 8529, 7804, 15678, 39507, 28229, 47523, 2732, 43671, 30348, 5661, 17698, 29351, 49082, 29480, 5822, 47699, 37300, 12736, 44355, 9852, 40759, 48826, 15901, 18971, 39392, 44485, 15853, 29463, 23665, 51552, 46796, 1749, 13686, 9086, 35259, 17499, 6380, 36669, 40709, 46506, 6210, 46089, 48113, 10249, 8825, 42596, 41894, 35355, 15331, 30046, 40264, 47925, 6442, 8131, 26203, 23702, 5626, 28279, 47362, 35974, 22959, 46591, 33876, 30744, 17324, 19369, 22123, 27885, 4663, 20597, 5649, 10874, 42536, 995, 1150, 27377, 43850, 50165, 36299, 43796, 17623, 16681, 8921, 16493, 33093, 35473, 3817, 40137, 5275, 44644, 19081, 29960, 40490, 34162, 18232, 5337, 36041, 27972, 12931, 14448, 17217, 10956, 31457, 51437, 7517, 39060, 35728, 43017, 18516, 51526, 49707, 50298, 7411, 16030, 1267, 10180, 9541, 49567, 51756, 24548, 51509, 19797, 3570, 10461, 23834, 21132, 15703, 16191, 20309, 51407, 13550, 43481, 935, 32837, 24165, 29887, 2367, 4407, 44865, 1272, 24491, 44706, 32047, 28993, 879, 30544, 30713, 23984, 21496, 19806, 38193, 40184, 4399, 48885, 5315, 42977, 31645, 41745, 12772, 18875, 14672, 13212, 31332, 6698, 44618, 19083, 11293, 10998, 33165, 15135, 49346, 10650, 22288, 1963, 28501, 19365, 34434, 13157, 29037, 35903, 34928, 8276, 10613, 21361, 7705, 20508, 29413, 9040, 16923, 14142, 18311, 796, 38967, 17675, 50513, 24052, 1140, 50545, 39139, 46269, 2035, 6781, 17109, 1163, 22719, 21757, 11256, 46911, 13587, 5114, 22742, 29103, 14682, 39617, 22370, 36750, 40648, 1497, 51247, 9647, 22446, 50634, 40291, 38091, 20171, 35242, 41267, 31206, 20868, 16632, 18210, 23578, 34135, 42096, 13050, 32129, 44322, 50593, 38515, 39759, 28902, 18104, 51474, 9477, 9361, 25591, 42130, 3062, 48218, 30880, 45540, 30843, 584, 47763, 2345, 24972, 48487, 43729, 23731, 22157, 20391, 29884, 46222, 30114, 31338, 51113, 34052, 17529, 5020, 30767, 31900, 4298, 42856, 42236, 50092, 20286, 38273, 8153, 32752, 46244, 3300, 16032, 701, 44526, 44759, 28431, 42381, 13475, 16233, 19015, 1980, 40041, 3389, 27631, 50398, 48394, 18102, 21091, 12626, 20361, 18395, 1175, 38796, 45285, 45173, 17279, 51020, 35882, 3524, 13867, 15136, 45368, 18340, 13630, 5699, 14998, 46624, 3129, 10557, 12813, 23203, 34626, 12735, 48430, 26370, 8947, 35603, 3506, 48275, 24981, 43345, 12424, 15063, 21587, 48411, 15809, 24887, 3282, 26461, 30585, 12388, 20339, 45431, 19727, 39555, 18254, 7531, 22309, 36108, 4834, 27901, 12816, 25005, 28781, 7033, 50970, 49967, 14320, 41050, 12274, 11900, 4502, 35682, 39610, 5433, 36964, 22455, 49224, 9115, 21331, 32221, 37019, 6628, 23420, 42611, 40960, 43219, 7630, 24428, 46142, 17506, 43051, 32465, 20375, 18714, 17331, 45407, 47047, 13571, 10794, 9908, 4725, 45553, 26790, 47091, 3346, 25541, 156, 51397, 10066, 35548, 42141, 6507, 41102, 23085, 3787, 34834, 26865, 19279, 14653, 19801, 34252, 16567, 30743, 20063, 46266, 11245, 27816, 117, 21342, 12803, 23473, 12718, 31305, 21620, 9715, 1789, 37255, 35560, 17739, 41532, 51559, 32764, 6879, 28535, 17299, 5013, 45814, 22737, 28013, 9034, 11929, 38324, 39639, 43095, 18448, 27079, 9457, 21159, 30910, 22506, 506, 21728, 49187, 36006, 25572, 21498, 30804, 34533, 5297, 4074, 19065, 25895, 22435, 51008, 24570, 31047, 4574, 11861, 19261, 42220, 20371, 25981, 11387, 23383, 18609, 31773, 32097, 43027, 4537, 36726, 35185, 2178, 28782, 9733, 32583, 44837, 33404, 30317, 22441, 12862, 34418, 5062, 49538, 6405, 11755, 28388, 10728, 43493, 16963, 24644, 46519, 34934, 39912, 36171, 39276, 43601, 39075, 43301, 15156, 43594, 46597, 26483, 10947, 557, 5845, 25612, 22768, 16614, 7759, 27179, 24872, 31970, 4963, 22256, 1320, 31779, 7361, 813, 16755, 12148, 34998, 9288, 48504, 14027, 37007, 48049, 36864, 49698, 38469, 11307, 42929, 46527, 41810, 9043, 36332, 20879, 35226, 48985, 5182, 51311, 26100, 25295, 17519, 24896, 50388, 42644, 21925, 12207, 24105, 11624, 8781, 37368, 23506, 34103, 23945, 12407, 40429, 42170, 20818, 29813, 8106, 50539, 41577, 12877, 37806, 25036, 18598, 35147, 50547, 35708, 24441, 48758, 14856, 44302, 20252, 26982, 25810, 34762, 40381, 20206, 37907, 5997, 41520, 18574, 24845, 3553, 51382, 16361, 9784, 8890, 25252, 14109, 45021, 14784, 31962, 502, 22797, 14408, 34082, 3873, 9028, 7975, 30596, 32582, 46293, 51237, 49065, 43464, 17086, 47383, 29136, 51564, 17116, 39349, 16703, 44414, 43059, 41970, 10258, 31314, 28924, 24222, 50754, 16251, 330, 49597, 4969, 36169, 39678, 16183, 30191, 15590, 21682, 3441, 38909, 7335, 12768, 37698, 39425, 46681, 24390, 26815, 5173, 13800, 23268, 20335, 34877, 41211, 4950, 27144, 12915, 25754, 6626, 46209, 8998, 10589, 3107, 18785, 36592, 25336, 19073, 33708, 36997, 41007, 33031, 9789, 22339, 30452, 29308, 31187, 27748, 45952, 15822, 34689, 11080, 8607, 23609, 29501, 44201, 29340, 39237, 39251, 29092, 12918, 1975, 51316, 46072, 44831, 32675, 25531, 30456, 23421, 6003, 37464, 46521, 51246, 49739, 11921, 17522, 15356, 5434, 5207, 33947, 6219, 33831, 43214, 5787, 5705, 25123, 33240, 11171, 37132, 605, 10988, 27000, 24425, 11226, 46798, 17444, 2558, 40021, 5828, 15872, 21137, 2411, 20029, 38293, 27047, 44198, 41853, 43877, 28029, 2657, 7580, 28674, 36028, 48659, 2602, 40671, 50711, 18745, 42942, 41300, 33175, 44465, 16447, 30193, 34079, 25468, 25941, 14136, 31223, 670, 35678, 23879, 28132, 34641, 44534, 4220, 20829, 46981, 44895, 36684, 46164, 25153, 24564, 48277, 26301, 6224, 40083, 8278, 27453, 45414, 4611, 30125, 30140, 36236, 5286, 29134, 11794, 38725, 1521, 25126, 40089, 17940, 3412, 34164, 8888, 29618, 17328, 15644, 6155, 48325, 37250, 20923, 7172, 18967, 20786, 33177, 15668, 40858, 23298, 20658, 68]
test_set_he = [23945, 12407, 40429, 42170, 20818, 29813, 8106, 50539, 41577, 12877, 37806, 25036, 18598, 35147, 50547, 35708, 24441, 48758, 14856, 44302, 20252, 26982, 25810, 34762, 40381, 20206, 37907, 5997, 41520, 18574, 24845, 3553, 51382, 16361, 9784, 8890, 25252, 14109, 45021, 14784, 31962, 502, 22797, 14408, 34082, 3873, 9028, 7975, 30596, 32582, 46293, 51237, 49065, 43464, 17086, 47383, 29136, 51564, 17116, 39349, 16703, 44414, 43059, 41970, 10258, 31314, 28924, 24222, 50754, 16251, 330, 49597, 4969, 36169, 39678, 16183, 30191, 15590, 21682, 3441, 38909, 7335, 12768, 37698, 39425, 46681, 24390, 26815, 5173, 13800, 23268, 20335, 34877, 41211, 4950, 27144, 12915, 25754, 6626, 46209, 8998, 10589, 3107, 18785, 36592, 25336, 19073, 33708, 36997, 41007, 33031, 9789, 22339, 30452, 29308, 31187, 27748, 45952, 15822, 34689, 11080, 8607, 23609, 29501, 44201, 29340, 39237, 39251, 29092, 12918, 1975, 51316, 46072, 44831, 32675, 25531, 30456, 23421, 6003, 37464, 46521, 51246, 49739, 11921, 17522, 15356, 5434, 5207, 33947, 6219, 33831, 43214, 5787, 5705, 25123, 33240, 11171, 37132, 605, 10988, 27000, 24425, 11226, 46798, 17444, 2558, 40021, 5828, 15872, 21137, 2411, 20029, 38293, 27047, 44198, 41853, 43877, 28029, 2657, 7580, 28674, 36028, 48659, 2602, 40671, 50711, 18745, 42942, 41300, 33175, 44465, 16447, 30193, 34079, 25468, 25941, 14136, 31223, 670, 35678, 23879, 28132, 34641, 44534, 4220, 20829, 46981, 44895, 36684, 46164, 25153, 24564, 48277, 26301, 6224, 40083, 8278, 27453, 45414, 4611, 30125, 30140, 36236, 5286, 29134, 11794, 38725, 1521, 25126, 40089, 17940, 3412, 34164, 8888, 29618, 17328, 15644, 6155, 48325, 37250, 20923, 7172, 18967, 20786, 33177, 15668, 40858, 23298, 20658, 68]

In [89]:
print(all_test_idxs)

[3980, 6711, 16982, 17926, 40191, 30227, 49118, 30548, 38271, 26759, 37694, 13913, 46710, 16674, 3675, 31906, 2799, 23876, 28466, 14035, 22012, 24387, 21521, 27939, 2043, 20603, 46368, 13704, 13272, 6016, 21698, 9891, 34977, 28076, 43124, 7811, 25055, 7802, 31354, 33302, 29795, 17481, 12953, 4649, 3915, 35666, 1603, 26121, 30990, 18703, 20933, 25410, 42204, 3793, 42671, 40175, 38941, 31135, 33771, 51594, 47777, 17539, 36980, 30799, 49750, 32934, 28548, 7996, 47107, 35794, 2899, 14790, 6182, 12253, 40727, 44471, 18961, 37568, 12287, 44793, 28804, 37067, 42720, 41095, 48656, 16375, 31386, 3630, 15057, 12354, 40577, 29961, 3772, 19445, 19350, 5299, 44799, 13585, 49913, 45969, 33316, 15360, 10379, 10025, 44823, 51293, 4153, 7911, 17200, 50700, 40110, 44546, 51065, 20887, 31595, 42465, 42534, 11718, 26180, 35937, 17069, 23034, 49238, 30352, 45784, 33509, 25016, 34009, 46116, 48723, 25519, 4215, 40015, 50611, 25007, 26655, 33264, 29305, 22967, 3766, 15157, 41910, 39068, 33448, 41697, 39971, 

In [88]:
# idxs per cluster
from collections import defaultdict
idxs_c=defaultdict(list)
idxc_c_test = defaultdict(list)
idxc_c_train = defaultdict(list)
idxc_c_train_balanced = defaultdict(list)
rng = np.random.RandomState(42)
cluster_probs_le = cluster_probs[low_entr_idxs].round()
# cluster_probs_hardened = cluster_probs.copy().round()
s = 0
all_train_idxs = []
all_test_idxs = []
all_train_balanced_idxs = []
for c in range(cluster_probs.shape[-1]):
    idxs_cluster = np.where(cluster_probs_le[:,c] == 1)[0]
    idxs_cluster = low_entr_idxs[idxs_cluster]
    s+=len(idxs_cluster)
    idxs_cluster_test = rng.choice(idxs_cluster, 150, replace=False)
    idxs_cluster_train = np.setdiff1d(idxs_cluster, idxs_cluster_test)
    if 1200 > len(idxs_cluster_train):
        idxs_cluster_train_balanced = rng.choice(idxs_cluster_train, len(idxs_cluster_train), replace=False)
        # idxs_cluster_train_balanced = np.concatenate([idxs_cluster_train_balanced, rng.choice(idxs_cluster_train, 1200-len(idxs_cluster_train), replace=False)])
    else:
        idxs_cluster_train_balanced = rng.choice(idxs_cluster_train, 1200, replace=False)
    print(c, "train:",len(idxs_cluster_train), "test:", len(idxs_cluster_test), "total:", len(idxs_cluster), "balanced:", len(idxs_cluster_train_balanced))
    
    idxs_c[c] = idxs_cluster.tolist()
    idxc_c_test[c] = idxs_cluster_test.tolist()
    idxc_c_train[c] = idxs_cluster_train.tolist()
    idxc_c_train_balanced[c] = idxs_cluster_train_balanced.tolist()
    all_train_idxs.extend(idxs_cluster_train.tolist())
    all_test_idxs.extend(idxs_cluster_test.tolist())
    all_train_balanced_idxs.extend(idxs_cluster_train_balanced.tolist())


# test high entropy examples
test_set_he = rng.choice(high_entr_idxs, 250, replace=False)
# train_set_he = np.setdiff1d(high_entr_idxs, test_set_he)
all_test_idxs = all_test_idxs + test_set_he.tolist()
# all_train_idxs


0 train: 769 test: 150 total: 919 balanced: 769
1 train: 2550 test: 150 total: 2700 balanced: 1200
2 train: 5457 test: 150 total: 5607 balanced: 1200
3 train: 1796 test: 150 total: 1946 balanced: 1200
4 train: 2438 test: 150 total: 2588 balanced: 1200
5 train: 2722 test: 150 total: 2872 balanced: 1200
6 train: 3968 test: 150 total: 4118 balanced: 1200
7 train: 1217 test: 150 total: 1367 balanced: 1200


In [90]:
#overlap
np.intersect1d(all_test_idxs, all_train_balanced_idxs)

array([], dtype=int64)

In [91]:
len(all_train_balanced_idxs)

9169

In [92]:
from mttl.dataloader.alpaca_dataset_readers import AlpacaDataset               
tokenizer = LlamaTokenizer.from_pretrained("yahma/llama-7b-hf", add_eos_token=True)
tokenizer.pad_token_id = 0 
max_input_length=256   
max_output_length=128 
data_dir=None          
dataset = AlpacaDataset(tokenizer, max_input_length, max_output_length, data_dir, train_on_inputs=False)    

100%|██████████| 1/1 [00:00<00:00, 673.57it/s]


In [93]:

cluster_infos_new = ClusterInfos()     
for i,(example, ex_dict_raw) in tqdm.tqdm(enumerate(zip(dataset,dataset.dataset)), total=len(dataset)):
    # topics_example = example["skill_set"]
    assert example.input_text.split("\n### Instruction:")[1].split("###")[0].strip() == ex_dict_raw["instruction"].strip(), (example.input_text.split("Instruction:")[1].split("###")[0].strip(), ex_dict_raw["instruction"])
    hash = example.hash      
    probs = example_to_distances[hash]
    index = example_to_indexes[hash]
    # print(index)
    # if index in low_entr_idxs:   
    #     probs = np.array(probs).round().tolist()
    #     assert index in idxs_c[np.argmax(probs)]
    is_test = int(index in all_test_idxs)
    
    if is_test and index in low_entr_idxs:   
        probs = np.array(probs).round().tolist()
        assert index in idxs_c[np.argmax(probs)]
    
    # if is_test==0 and index in all_train_balanced_idxs:
    #     is_test = -1
    if is_test==1 and not index in test_set_he:
        assert index in idxc_c_test[np.argmax(probs)]
        # print("test", index)
    assert np.sum(probs)==1
    # print(np.sum(probs))    
    main_t = np.argmax(probs)
    cluster_infos_new.is_test.extend([is_test])     
    cluster_infos_new.task_names.extend([str])
    cluster_infos_new.cluster_ids.extend([int(main_t)])    
    cluster_infos_new.hashes.extend([hash])
    cluster_infos_new.cluster_dists.extend([probs])
# assert sum(cluster_infos_new.is_test) == len(all_test_idxs)

100%|██████████| 51760/51760 [01:33<00:00, 553.25it/s]


In [94]:
np.where(np.array(cluster_infos_new.is_test)==-1)[0].shape

(0,)

In [95]:
sum(np.array(cluster_infos_new.is_test)==1) == len(all_test_idxs)

True

In [96]:
idxs_c0 = np.where(np.array(cluster_infos_new.cluster_dists)[:,0]==1)[0]
idxs_test = np.where(np.array(cluster_infos_new.is_test)==1)[0]

In [97]:
#overlap
len(np.intersect1d(idxs_c0, idxs_test))

150

In [98]:
# cluster_infos_new.save("/home/v-oostapenko/dev/mttl/inst_follow/cluster_infos/atlas_by_instr_text-embedding-ada-002_ldalayer1_hardened<01+test_le.pkl")
# cluster_infos_new.save("/home/v-oostapenko/dev/mttl/inst_follow/cluster_infos/atlas_by_instr_text-embedding-ada-002_ldalayer1_hardened_01_test_le_balanced.pkl")
cluster_infos_new.save("/home/v-oostapenko/dev/mttl/inst_follow/cluster_infos/atlas_by_instr_text-embedding-ada-002_ldalayer1_test.pkl")

### hardened + tes_le

In [326]:
# hardened + tes_le
#harden all
cluster_probs_hardened = np.array(cluster_infos.cluster_dists.copy())
argmaxidx = np.argmax(np.array(cluster_infos.cluster_dists.copy()),-1)
# set argmax idx to 1
cluster_probs_hardened = np.zeros_like(cluster_probs_hardened)
cluster_probs_hardened[np.arange(len(cluster_probs_hardened)), argmaxidx] = 1
# unharden the he test set
cluster_probs_hardened[test_set_he] = np.array(cluster_infos.cluster_dists)[test_set_he]
example_to_distances = dict(
            zip(
                cluster_infos.hashes,     
                list(cluster_probs_hardened),
            )
)


In [327]:
for ti in test_set_he[:10]:
    print(cluster_probs_hardened[ti][:8])

[0.      0.15625 0.125   0.      0.0625  0.0625  0.125   0.46875]
[0.      0.09375 0.0625  0.28125 0.34375 0.0625  0.125   0.03125]
[0.      0.03125 0.3125  0.3125  0.03125 0.      0.3125  0.     ]
[0.      0.03125 0.25    0.34375 0.03125 0.      0.34375 0.     ]
[0.      0.4375  0.03125 0.03125 0.28125 0.      0.21875 0.     ]
[0.      0.09375 0.34375 0.40625 0.03125 0.      0.125   0.     ]
[0.      0.28125 0.      0.21875 0.0625  0.40625 0.      0.03125]
[0.      0.21875 0.34375 0.0625  0.03125 0.      0.34375 0.     ]
[0.03125 0.1875  0.      0.5     0.21875 0.      0.0625  0.     ]
[0.      0.34375 0.      0.21875 0.34375 0.      0.09375 0.     ]


In [328]:

from mttl.dataloader.alpaca_dataset_readers import AlpacaDataset
tokenizer = LlamaTokenizer.from_pretrained("yahma/llama-7b-hf", add_eos_token=True)
tokenizer.pad_token_id = 0 
max_input_length=256   
max_output_length=128 
data_dir=None          
dataset = AlpacaDataset(tokenizer, max_input_length, max_output_length, data_dir, train_on_inputs=False)    
cluster_infos_new = ClusterInfos()           
for i,(example, ex_dict_raw) in tqdm.tqdm(enumerate(zip(dataset,dataset.dataset)), total=len(dataset)):
    # topics_example = example["skill_set"]
    assert example.input_text.split("\n### Instruction:")[1].split("###")[0].strip() == ex_dict_raw["instruction"].strip(), (example.input_text.split("Instruction:")[1].split("###")[0].strip(), ex_dict_raw["instruction"])
    hash = example.hash      
    probs = example_to_distances[hash]
    index = example_to_indexes[hash]
    is_test = int(index in all_test_idxs)
    assert np.sum(probs)==1
    # print(np.sum(probs))    
    main_t = np.argmax(probs)
    cluster_infos_new.is_test.extend([is_test])     
    cluster_infos_new.task_names.extend([str])
    cluster_infos_new.cluster_ids.extend([int(main_t)])    
    cluster_infos_new.hashes.extend([hash])
    cluster_infos_new.cluster_dists.extend([probs])
# assert sum(cluster_infos_new.is_test) == len(all_test_idxs)

100%|██████████| 51760/51760 [01:27<00:00, 594.03it/s]


In [329]:
sum(cluster_infos_new.is_test) == len(all_test_idxs)  

True

In [330]:
cluster_infos_new.save("/home/v-oostapenko/dev/mttl/inst_follow/cluster_infos/atlas_by_instr_text-embedding-ada-002_ldalayer1_hardened+tes_le.pkl")
# cluster_infos_new.save("/home/v-oostapenko/dev/mttl/inst_follow/cluster_infos/atlas_by_instr_text-embedding-ada-002_ldalayer1_hardened<01+hardened_dist+test_le.pkl")
# cluster_infos_new.save("/home/v-oostapenko/dev/mttl/inst_follow/cluster_infos/atlas_by_instr_text-embedding-ada-002_ldalayer1_hardened<01+test_le.pkl")
# cluster_infos_new.save("/home/v-oostapenko/dev/mttl/inst_follow/cluster_infos/atlas_by_instr_text-embedding-ada-002_ldalayer1_hardened<01+dist+test_le.pkl")

###  hardened_dist + test_le

In [ ]:
# for low-entropy examples, we add them to several clusters, to all clusters if p>0.1


In [ ]:
sum(cluster_infos.is_test) == len(all_test_idxs)

False

In [ ]:
sum(cluster_infos.is_test)

1442